# 7.5 IMDb（Internet Movie Database）からDataLoaderを作成

- 本ファイルでは、IMDb（Internet Movie Database）のデータを使用して、感情分析（0：ネガティブ、1：ポジティブ）を2値クラス分類するためのDatasetとDataLoaderを作成します。


※　本章のファイルはすべてUbuntuでの動作を前提としています。Windowsなど文字コードが違う環境での動作にはご注意下さい。

# 7.5 学習目標

1.	テキスト形式のファイルデータからtsvファイルを作成し、torchtext用のDataLoaderを作成できるようになる

# 事前準備
書籍の指示に従い、本章で使用するデータを用意します


# 1. IMDbデータセットをtsv形式に変換

Datasetをダウンロードします

※torchtextで標準でIMDbが使える関数があるのですが、今回は今後データセットが用意されていない場合でも対応できるように0から作ります。

http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

5万件のデータ（train,testともに2.5万件）です。データidとrating（1-10）でファイル名が決まっています。

rateは10の方が良いです。4以下がnegative、7以上がpositiveにクラス分けされています。



In [1]:
# tsv形式のファイルにします
import glob
import os
import io
import string


# 訓練データのtsvファイルを作成します

f = open('./data/IMDb_train.tsv', 'w')

path = './data/aclImdb/train/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)

path = './data/aclImdb/train/neg/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)

f.close()


In [2]:
# テストデータの作成

f = open('./data/IMDb_test.tsv', 'w')

path = './data/aclImdb/test/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)


path = './data/aclImdb/test/neg/'

for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)

f.close()


# 2. 前処理と単語分割の関数を定義

In [15]:
import string
import re

# 以下の記号はスペースに置き換えます（カンマ、ピリオドを除く）。
# punctuationとは日本語で句点という意味です
print("区切り文字：", string.punctuation)
# !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

# 前処理


def preprocessing_text(text):
    # 改行コードを消去
    text = re.sub('<br />', '', text)

    # カンマ、ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")

    # ピリオドなどの前後にはスペースを入れておく
    text = text.replace(".", " . ")
    text = text.replace(",", " , ")
    return text

# 分かち書き（今回はデータが英語で、簡易的にスペースで区切る）


def tokenizer_punctuation(text):
    return text.strip().split()


# 前処理と分かち書きをまとめた関数を定義
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    ret = tokenizer_punctuation(text)
    return ret


# 動作を確認します
print(tokenizer_with_preprocessing('I like cat.'))


区切り文字： !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['I', 'like', 'cat', '.']


# DataLoaderの作成

In [5]:
# データを読み込んだときに、読み込んだ内容に対して行う処理を定義します
import torchtext


# 文章とラベルの両方に用意します
max_length = 50
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True,
                            lower=True, include_lengths=True, batch_first=True, fix_length=max_length, init_token="<cls>", eos_token="<eos>")
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

# 引数の意味は次の通り
# init_token：全部の文章で、文頭に入れておく単語
# eos_token：全部の文章で、文末に入れておく単語


In [6]:
# フォルダ「data」から各tsvファイルを読み込みます
train_val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='IMDb_train.tsv',
    test='IMDb_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

# 動作確認
print('訓練および検証のデータ数', len(train_val_ds))
print('1つ目の訓練および検証のデータ', vars(train_val_ds[0]))


訓練および検証のデータ数 25000
1つ目の訓練および検証のデータ {'Text': ['i', 'couldn', 't', 'believe', 'the', 'comments', 'made', 'about', 'the', 'movie', 'as', 'i', 'read', 'the', 'awful', 'opinions', 'about', 'the', 'movie', 'i', 'actually', 'wondered', 'if', 'you', 'had', 'actually', 'viewed', 'the', 'same', 'movie', 'that', 'i', 'did', 'what', 'i', 'viewed', 'was', 'incredible', 'i', 'think', 'the', 'actresses', 'and', 'director', 'did', 'a', 'fantastic', 'job', 'in', 'the', 'movie', 'i', 'hadn', 't', 'had', 'the', 'pleasure', 'to', 'see', 'either', 'actress', 'previously', 'and', 'i', 'couldn', 't', 'have', 'been', 'more', 'set', 'back', 'by', 'the', 'incredible', 'job', 'that', 'they', 'did', 'i', 'd', 'have', 'to', 'say', 'its', 'the', 'most', 'believable', 'movie', 'that', 'i', 've', 'seen', 'in', 'a', 'long', 'time', 'what', 'i', 'don', 't', 'see', 'is', 'why', 'everyone', 'has', 'such', 'a', 'problem', 'with', 'deanna', 's', 'choice', 'of', 'drug', 'in', 'the', 'attempt', 'of', 'suicide', 'scene', 'fro

In [7]:
import random
# torchtext.data.Datasetのsplit関数で訓練データとvalidationデータを分ける

train_ds, val_ds = train_val_ds.split(
    split_ratio=0.8, random_state=random.seed(1234))

# 動作確認
print('訓練データの数', len(train_ds))
print('検証データの数', len(val_ds))
print('1つ目の訓練データ', vars(train_ds[0]))


訓練データの数 20000
検証データの数 5000
1つ目の訓練データ {'Text': ['i', 'watched', 'the', 'entire', 'movie', 'recognizing', 'the', 'participation', 'of', 'william', 'hurt', 'natascha', 'mcelhone', 'and', 'desiree', 'nosbusch', 'i', 'm', 'glad', 'that', 'i', 'had', 'no', 'idea', 'of', 'the', 'presence', 'of', 'peter', 'weller', 'at', 'the', 'end', 'of', 'the', 'movie', 'i', 'said', 'that', 'was', 'peter', 'weller', 'kudos', 'to', 'mr', 'weller', 'for', 'an', 'outstanding', 'performance', 'weller', 'played', 'a', 'major', 'character', 'and', 'his', 'performance', 'was', 'such', 'that', 'i', 'didn', 't', 'even', 'recognize', 'him', 'overall', 'the', 'plot', 'was', 'bad', 'the', 'writing', 'was', 'bad', 'and', 'the', 'performances', 'aside', 'from', 'those', 'of', 'nosbusch', 'and', 'weller', 'were', 'subpar', 'the', 'scenery', 'and', 'setting', 'were', 'interesting', 'and', 'weller', 'was', 'amazing', '4', 'stars', 'of', 'a', 'possible', '10'], 'Label': '0'}


# ボキャブラリーを作成

In [8]:
# torchtextで単語ベクトルとして英語学習済みモデルを読み込みます

from torchtext.vocab import Vectors

english_fasttext_vectors = Vectors(name='data/wiki-news-300d-1M.vec')


# 単語ベクトルの中身を確認します
print("1単語を表現する次元数：", english_fasttext_vectors.dim)
print("単語数：", len(english_fasttext_vectors.itos))


1単語を表現する次元数： 300
単語数： 999994


In [9]:
# ベクトル化したバージョンのボキャブラリーを作成します
TEXT.build_vocab(train_ds, vectors=english_fasttext_vectors, min_freq=10)

# ボキャブラリーのベクトルを確認します
print(TEXT.vocab.vectors.shape)  # 17903個の単語が300次元のベクトルで表現されている
TEXT.vocab.vectors

# ボキャブラリーの単語の順番を確認します
TEXT.vocab.stoi


torch.Size([17903, 300])


defaultdict(<function torchtext.vocab._default_unk_index()>,
            {'<unk>': 0,
             '<pad>': 1,
             '<cls>': 2,
             '<eos>': 3,
             'the': 4,
             'and': 5,
             'a': 6,
             'of': 7,
             'to': 8,
             'is': 9,
             'it': 10,
             'in': 11,
             'i': 12,
             'this': 13,
             'that': 14,
             's': 15,
             'was': 16,
             'as': 17,
             'for': 18,
             'with': 19,
             'movie': 20,
             'but': 21,
             'film': 22,
             't': 23,
             'you': 24,
             'on': 25,
             'not': 26,
             'he': 27,
             'are': 28,
             'his': 29,
             'have': 30,
             'be': 31,
             'one': 32,
             'all': 33,
             'at': 34,
             'they': 35,
             'by': 36,
             'who': 37,
             'an': 38,
             'fro

In [ ]:
# DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
train_dl = torchtext.data.Iterator(train_ds, batch_size=24, train=True)

val_dl = torchtext.data.Iterator(
    val_ds, batch_size=24, train=False, sort=False)

test_dl = torchtext.data.Iterator(
    test_ds, batch_size=24, train=False, sort=False)


# 動作確認 検証データのデータセットで確認
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)


このようにDataLoaderは単語のidを格納しているので、分散表現はディープラーニングモデル側でidに応じて取得してあげる必要があります。

ここまでの内容をフォルダ「utils」のdataloader.pyに別途保存しておき、次節からはこちらから読み込むようにします

以上